In [8]:
import os
import json
import spacy
from tqdm.notebook import tqdm
from spacy.scorer import Scorer
from spacy.training import Example

In [9]:
gpu = spacy.prefer_gpu()
print(gpu)

True


In [10]:
nlp = spacy.load('../training-output/v2/model-best')

In [11]:
with open('../label-studio-files/v1/output.json') as f:
    labeled_data = json.load(f)

In [12]:
training_labeled_data = []
for data in labeled_data:
    text = data["data"]["text"]
    entities = []
    for annotation in data["annotations"][0]["result"]:
        start = annotation["value"]["start"]
        end = annotation["value"]["end"]
        label = annotation["value"]["labels"][0]
        for ent in entities:
            # check if the current entity is subspan of the annotated entity
            if (start >= ent[0] and end <= ent[1]) or (start <= ent[0] and end >= ent[1]):
                break
        else:
            entities.append((start, end, label))
    training_labeled_data.append((text, {"entities": entities}))

In [13]:
scorer = Scorer(nlp)

In [15]:
job_descriptions = []
for text, entities in training_labeled_data:
    predicted = nlp(text)
    example = Example.from_dict(predicted, entities)
    job_descriptions.append(example)
scorer.score(job_descriptions)

/Users/iamvinitk/miniconda3/envs/NER-CMPE-257/lib/python3.8/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Airbnb was born in 2007 when two Hosts welcomed th..." with entities "[(404, 414, 'SOFT_SKILL'), (599, 612, 'TECH_SKILL'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/iamvinitk/miniconda3/envs/NER-CMPE-257/lib/python3.8/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "About Us
At Cloudflare, we have our eyes set on an..." with entities "[(381, 389, 'TECH_SKILL'), (1407, 1412, 'TECH_SKIL...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/iamvinitk/miniconda3/envs/NER-CMPE-257/lib/py

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.6464367816091954,
 'ents_r': 0.5103448275862069,
 'ents_f': 0.5703853955375254,
 'ents_per_type': {'ROLE': {'p': 0.68,
   'r': 0.5396825396825397,
   'f': 0.6017699115044247},
  'SOFT_SKILL': {'p': 0.6253521126760564,
   'r': 0.7375415282392026,
   'f': 0.6768292682926829},
  'TECH_SKILL': {'p': 0.6204595997034841,
   'r': 0.5113011606597434,
   'f': 0.5606162089752178},
  'DEGREE': {'p': 0.4732824427480916, 'r': 0.5299145299145299, 'f': 0.5},
  'PROG_LANG': {'p': 0.9041666666666667,
   'r': 0.8509803921568627,
   'f': 0.8767676767676768},
  'REQ': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'RESP': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'OTHER': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'DOMAIN': {'p': 0.0, 'r': 0.0, 'f': 0.0}}}

In [16]:
from itertools import groupby


def doc_to_spans(doc):
    tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
    results = []
    entities = set()
    for entity, group in groupby(tokens, key=lambda t: t[-1]):
        if not entity:
            continue
        group = list(group)
        _, start, _ = group[0]
        word, last, _ = group[-1]
        text = ' '.join(item[0] for item in group)
        end = last + len(word)
        results.append({
            'from_name': 'label',
            'to_name': 'text',
            'type': 'labels',
            'value': {
                'start': start,
                'end': end,
                'text': text,
                'labels': [entity]
            }
        })
        entities.add(entity)

    return results, entities

In [17]:
entities = set()
tasks = []
for data in labeled_data:
    text = data["data"]["text"]
    doc = nlp(text)
    spans, ents = doc_to_spans(doc)
    entities |= ents

    tasks.append({
        'data': {'text': text},
        'predictions': [{'model_version': 'v2', 'result': spans}],
    })

In [18]:
# Save Label Studio tasks.json
print(f'Save {len(tasks)} tasks to "tasks.json"')
with open('../label-studio-files/v2/tasks.json', mode='w') as f:
    json.dump(tasks, f, indent=2)
    
# Save class labels as a txt file
print('Named entities are saved to "named_entities.txt"')
with open('../label-studio-files/v2/named_entities.txt', mode='w') as f:
    f.write('\n'.join(sorted(entities)))

Save 121 tasks to "tasks.json"
Named entities are saved to "named_entities.txt"


### References
1. https://labelstud.io/blog/evaluating-named-entity-recognition-parsers-with-spacy-and-label-studio/